In [9]:
import numpy as np
from itertools import combinations
import time

In [2]:
def make_deck():
    return [1,2,3,4,5,6,7,8,9,10,10,10,10]*4

In [3]:
def create_game():
    deck = make_deck()
    np.random.shuffle(deck)
    hand = deck[:7]
    val = list(filter((10).__ne__, deck[7:]))[:3]
    return hand, 100*val[0] + 10*val[1] + val[2]

In [4]:
ops = ['+', '-', '*', '/']
inverse_ops = {'+':'-', '-':'+', '*':'/', '/':'x'}

def solver(hand, val):
    if len(hand) == 0 and val == 0:
        return True, []
    hand_c = hand.copy()
    for card in hand_c:
        hand.remove(card)
        for op in ops:
            if op == '/':
                w, l = solver(hand, val / card)
            elif op == '-':
                w, l = solver(hand, val - card)
            elif op == '+':
                w, l = solver(hand, val + card)
            else:
                w, l = solver(hand, val * card)
            if w:
                l.append([card, op])
                return True, l
        hand.append(card)
    return False, []

In [5]:
def pretty_print(sol):
    pretty = str(sol[0][0]) if sol[0][1] == '-' else str(-sol[0][0])
    for i, (card, op) in enumerate(sol[1:]):
        if i == 0:
            pretty = pretty + ' ' + inverse_ops[op] + ' ' + str(card)
        else:    
            pretty = '(' + pretty + ') ' + inverse_ops[op] + ' ' + str(card)
    return pretty

In [6]:
def play(verbose = True):
    h, v = create_game()
    if verbose:
        print(str(v) + ': ' + str(h))
    status, sol = solver(h, v)
    if verbose:
        if status:
            print('Solution: ' + pretty_print(sol))
        else:
            print('No Solution Found')
        return
    else:
        return status, sol, h, v

In [37]:
N = 100
count = 0
for _ in range(N):
    status, sol, h, v = play(False)
    if status:
        count += 1
    else:
        print(str(v) + ': ' + str(h))
print(count/N)

795: [10, 9, 6, 3, 10, 10, 10]
293: [3, 5, 2, 1, 1, 1, 2]
345: [10, 10, 10, 10, 8, 7, 10]
744: [10, 10, 9, 10, 10, 7, 10]
0.96


In [38]:
N = 3000
count = 0
for _ in range(N):
    status, sol, h, v = play(False)
    if status:
        count += 1
    else:
        print(str(v) + ': ' + str(h))
print(count/N)

732: [10, 10, 5, 2, 2, 10, 10]
857: [4, 10, 4, 10, 10, 1, 4]
833: [10, 4, 10, 10, 10, 10, 1]
615: [3, 10, 1, 10, 10, 10, 7]
429: [10, 10, 8, 10, 10, 10, 7]
572: [10, 10, 4, 1, 10, 10, 1]
444: [10, 10, 9, 10, 10, 9, 10]
317: [6, 10, 4, 10, 10, 6, 10]
952: [10, 10, 10, 6, 5, 10, 10]
924: [7, 1, 5, 1, 3, 5, 1]
847: [10, 10, 2, 8, 4, 6, 6]
858: [1, 2, 2, 10, 9, 1, 3]
897: [6, 10, 4, 10, 8, 8, 6]
936: [10, 10, 10, 7, 5, 10, 10]
413: [10, 10, 10, 10, 5, 10, 6]
938: [10, 10, 6, 5, 10, 1, 10]
766: [1, 10, 10, 10, 10, 10, 10]
782: [10, 5, 10, 10, 10, 9, 1]
365: [10, 10, 10, 6, 3, 10, 10]
931: [10, 10, 10, 9, 9, 9, 1]
577: [10, 1, 10, 9, 10, 1, 10]
346: [10, 7, 9, 10, 10, 10, 8]
924: [7, 10, 10, 9, 2, 10, 10]
423: [2, 2, 4, 10, 10, 10, 10]
837: [1, 8, 1, 10, 6, 1, 10]
834: [10, 10, 10, 2, 10, 7, 10]
931: [10, 2, 4, 8, 5, 2, 1]
943: [4, 10, 10, 5, 10, 1, 5]
527: [1, 9, 1, 8, 1, 9, 9]
862: [5, 10, 10, 10, 4, 1, 10]
563: [10, 4, 2, 9, 4, 2, 4]
731: [6, 2, 10, 4, 4, 4, 8]
882: [6, 10, 6, 10, 1, 10, 

In [15]:
def brute(hand, val):
    if len(hand) == 1 and val == hand[0]:
        return True, []
    hand_c = hand.copy()
    combos = set(combinations(hand_c, 2))
    for card2, card1 in combos:
        hand.remove(card1)
        hand.remove(card2)
        for op in ops:
            if op == '*':
                hand.append(card1 * card2)
                w, l = brute(hand, val)
                hand.remove(card1 * card2)
            elif op == '+':
                hand.append(card1 + card2)
                w, l = brute(hand, val)
                hand.remove(card1 + card2)
            elif op == '-':
                hand.append(card1 - card2)
                w, l = brute(hand, val)
                hand.remove(card1 - card2)
            elif not card2 == 0:
                hand.append(card1 / card2)
                w, l = brute(hand, val)
                hand.remove(card1 / card2)
            else:
                continue
            if w:
                hand.append(card1)
                hand.append(card2)
                l.append([card1, card2, op])
                return True, l
        hand.append(card1)
        hand.append(card2)
    return False, []

In [17]:
start = time.time()
print(brute([10, 2, 10, 10, 10, 8, 10], 791))
print(time.time() - start)

(True, [[1.0, 790, '+'], [10, 10, '/'], [780, 10, '+'], [78, 10, '*'], [80, 2, '-'], [8, 10, '*']])
0.5197689533233643


In [18]:
start = time.time()
print(brute([4, 1, 10, 5, 10, 1, 10], 733))
print(time.time() - start)

(False, [])
40.91533827781677


In [27]:
tot = 0
count = 0
sum_time = 0
with open('simple_unsolved.txt', 'r') as file:
    for line in file:
        tot += 1
        if line[-1] == '\n':
            sp = line[:-1].split(': ')
        else:
            sp = line.split(': ')
        start = time.time()
        status, sol = brute(eval(sp[1]), int(sp[0]))
        end = time.time()
        tot_time = end - start
        sum_time += tot_time
        if status:
            count += 1
            print('(' + sp[0] + ', ' + sp[1] + '): ' + str(sol) + ' (' + str(tot_time) + ')')
        else:
            print('(' + sp[0] + ', ' + sp[1] + '): No Solution' + ' (' + str(tot_time) + ')')
print('Average Time: ' + str(sum_time/tot))
print('Solved Percentage: ' + str(count/tot))

(732, [10, 10, 5, 2, 2, 10, 10]): [[720, 12, '+'], [60, 12, '*'], [10, 2, '+'], [50, 10, '+'], [10, 2, '+'], [5, 10, '*']] (1.2879889011383057)
(857, [4, 10, 4, 10, 10, 1, 4]): No Solution (17.536702871322632)
(833, [10, 4, 10, 10, 10, 10, 1]): No Solution (5.413914203643799)
(615, [3, 10, 1, 10, 10, 10, 7]): [[6.15, 100, '*'], [0.15, -6.0, '-'], [3, 20, '/'], [1, 7.0, '-'], [10, 10, '*'], [10, 10, '+']] (0.206406831741333)
(429, [10, 10, 8, 10, 10, 10, 7]): No Solution (6.450922012329102)
(572, [10, 10, 4, 1, 10, 10, 1]): No Solution (12.430142879486084)
(444, [10, 10, 9, 10, 10, 9, 10]): No Solution (4.705357074737549)
(317, [6, 10, 4, 10, 10, 6, 10]): [[15.85, 20, '*'], [16, 0.15, '-'], [10, 6, '+'], [6, 40, '/'], [10, 10, '+'], [10, 4, '*']] (1.846372127532959)
(952, [10, 10, 10, 6, 5, 10, 10]): [[940, 12.0, '+'], [94, 10, '*'], [100, 6, '-'], [10, 10, '*'], [2.0, 10, '+'], [10, 5, '/']] (1.869927167892456)
(924, [7, 1, 5, 1, 3, 5, 1]): No Solution (41.99949407577515)
(847, [10, 10

(718, [4, 10, 10, 10, 10, 10, 9]): [[720, 2.0, '-'], [36, 20, '*'], [20, 10, '/'], [10, 10, '+'], [10, 10, '+'], [9, 4, '*']] (1.3140077590942383)
(138, [6, 10, 10, 10, 10, 10, 9]): [[140, 2.0, '-'], [150, 10, '-'], [20, 10, '/'], [10, 10, '+'], [15, 10, '*'], [9, 6, '+']] (0.011726140975952148)
(637, [2, 2, 1, 2, 1, 10, 2]): No Solution (12.505697011947632)
(168, [10, 10, 10, 10, 10, 10, 10]): No Solution (0.21108603477478027)
(925, [3, 10, 10, 9, 10, 10, 10]): [[927, 2.0, '-'], [9, 103, '*'], [20, 10, '/'], [100, 3, '+'], [10, 10, '*'], [10, 10, '+']] (0.17162322998046875)
(742, [10, 10, 6, 10, 6, 10, 9]): [[10.6, 70, '*'], [0.6, 10, '+'], [7, 10, '*'], [10, 3, '-'], [6, 10, '/'], [9, 6, '-']] (0.34827327728271484)
(651, [3, 4, 5, 10, 10, 10, 10]): [[665, 14, '-'], [7, 95, '*'], [10, 3, '-'], [100, 5, '-'], [10, 10, '*'], [10, 4, '+']] (0.20950603485107422)
(975, [2, 2, 6, 4, 2, 5, 2]): No Solution (24.49433183670044)
(569, [10, 8, 10, 10, 4, 2, 10]): No Solution (30.15988302230835)


In [36]:
N = 500
count_q = 0
time_q = [0,0]
count_b = 0
time_b = [0,0]
for _ in range(N):
    h, v = create_game()
    start = time.time()
    status, sol = solver(h.copy(), v)
    end = time.time()
    if status:
        count_q += 1
        time_q[0] += (end - start)
    else:
        time_q[1] += (end - start)
    start = time.time()
    status, sol = brute(h, v)
    end = time.time()
    if status:
        count_b += 1
        time_b[0] += (end - start)
    else:
        time_b[1] += (end - start)
    
print('Q Percentage: ' + str(count_q/N))
if not count_q == 0:
    print('Q Solve Time: ' + str(time_q[0]/count_q))
if not N - count_q == 0:
    print('Q Fail Time: ' + str(time_q[1]/(N - count_q)))
print('Q Average Time: ' + str((time_q[0] + time_q[1])/N))
print('B Percentage: ' + str(count_b/N))
if not count_b == 0:
    print('B Solve Time: ' + str(time_b[0]/count_b))
if not N - count_b == 0:
    print('B Fail Time: ' + str(time_b[1]/(N - count_b)))
print('B Average Time: ' + str((time_b[0] + time_b[1])/N))

Q Percentage: 0.976
Q Solve Time: 1.6782688444755116
Q Fail Time: 38.17652412255605
Q Average Time: 2.5542269711494447
B Percentage: 0.992
B Solve Time: 0.6905140487417099
B Fail Time: 37.70773404836655
B Average Time: 0.9866518087387085


In [38]:
brute([10,10,10,10,1,1,3], 736)

(False, [])

In [51]:
sv = {}
def brute_speed(hand, val):
    if len(hand) == 1 and val == hand[0]:
        return True, []
    hand.sort()
    key = str(hand)
    #key = [0 if i == 10 else i for i in hand]
    #key = sum([key[i]*(10**i) for i in range(len(key))])
    if sv.get(key, False):
        return False, []
    hand_c = hand.copy()
    combos = set(combinations(hand_c, 2))
    for card2, card1 in combos:
        hand.remove(card1)
        hand.remove(card2)
        for op in ops:
            if op == '*':
                hand.append(card1 * card2)
                w, l = brute_speed(hand, val)
                hand.remove(card1 * card2)
            elif op == '+':
                hand.append(card1 + card2)
                w, l = brute_speed(hand, val)
                hand.remove(card1 + card2)
            elif op == '-':
                hand.append(card1 - card2)
                w, l = brute_speed(hand, val)
                hand.remove(card1 - card2)
            elif not card2 == 0:
                hand.append(card1 / card2)
                w, l = brute_speed(hand, val)
                hand.remove(card1 / card2)
            else:
                continue
            if w:
                hand.append(card1)
                hand.append(card2)
                l.append([card1, card2, op])
                return True, l
        hand.append(card1)
        hand.append(card2)
    sv[key] = True
    return False, []

In [53]:
N = 500
count_q = 0
time_q = [0,0]
count_b = 0
time_b = [0,0]
for _ in range(N):
    sv = {}
    h, v = create_game()
    start = time.time()
    status, sol = brute_speed(h.copy(), v)
    end = time.time()
    if status:
        count_q += 1
        time_q[0] += (end - start)
    else:
        time_q[1] += (end - start)
    start = time.time()
    status, sol = brute(h, v)
    end = time.time()
    if status:
        count_b += 1
        time_b[0] += (end - start)
    else:
        time_b[1] += (end - start)
    
print('Q Percentage: ' + str(count_q/N))
if not count_q == 0:
    print('Q Solve Time: ' + str(time_q[0]/count_q))
if not N - count_q == 0:
    print('Q Fail Time: ' + str(time_q[1]/(N - count_q)))
print('Q Average Time: ' + str((time_q[0] + time_q[1])/N))
print('B Percentage: ' + str(count_b/N))
if not count_b == 0:
    print('B Solve Time: ' + str(time_b[0]/count_b))
if not N - count_b == 0:
    print('B Fail Time: ' + str(time_b[1]/(N - count_b)))
print('B Average Time: ' + str((time_b[0] + time_b[1])/N))

Q Percentage: 0.988
Q Solve Time: 0.08379650646858369
Q Fail Time: 0.5273741881052653
Q Average Time: 0.08911943864822387
B Percentage: 0.988
B Solve Time: 0.5994911338636267
B Fail Time: 25.365475455919903
B Average Time: 0.896682945728302


In [56]:
0.5994911338636267/0.08379650646858369, 25.365475455919903/0.5273741881052653, 0.896682945728302/0.08911943864822387

(7.154130394306869, 48.09768097876054, 10.06158655540603)

In [105]:
def opstr(p1, p2, op, par_red):
    if par_red:
        if op == '*':
            return p1 + ' ' + op + ' ' + p2
        else:
            return '(' + p1 + ' ' + op + ' ' + p2 + ')'
    else:
        return '(' + p1 + ' ' + op + ' ' + p2 + ')'

def evalop(c1, c2, op):
    if op == '+':
        r = c1 + c2
    elif op == '-':
        r = c1 - c2
    elif op == '*':
        r = c1 * c2
    else:
        r = c1 / c2
    return r
    
def brute_pretty_print(sol, hand, par_red = True):
    if not sol:
        return 'No Solution Found'
    stor = {}
    for card in hand:
        if stor.get(card, False):
            stor[card].append(str(card))
        else:
            stor[card] = [str(card)]
    while sol:
        c1, c2, op = sol.pop()
        if c1 == c2:
            if stor.get(c1, False):
                if len(stor[c1]) > 1:
                    p1 = stor[c1][0]
                    stor[c1] = stor[c1][1:]
                    p2 = stor[c2][0]
                    if len(stor[c2]) > 1:
                        stor[c2] = stor[c2][1:]
                    else:
                        stor.pop(c2)
                    r = evalop(c1, c2, op)
                    if stor.get(r, False):
                        stor[r].append(opstr(p1, p2, op, par_red))
                    else:
                        stor[r] = [opstr(p1, p2, op, par_red)]
                else:
                   sol.insert(1, [c1, c2, op]) 
            else:
                sol.insert(1, [c1, c2, op])
        else:
            if stor.get(c1, False) and stor.get(c2, False):
                p1 = stor[c1][0]
                if len(stor[c1]) > 1:
                    stor[c1] = stor[c1][1:]
                else:
                    stor.pop(c1)
                p2 = stor[c2][0]
                if len(stor[c2]) > 1:
                    stor[c2] = stor[c2][1:]
                else:
                    stor.pop(c2)
                r = evalop(c1, c2, op)
                if stor.get(r, False):
                    stor[r].append(opstr(p1, p2, op, par_red))
                else:
                    stor[r] = [opstr(p1, p2, op, par_red)]
            else:
                sol.insert(1, [c1, c2, op])
    return stor[list(stor.keys())[0]][0][1:-1]

In [66]:
brute_speed(*create_game())

(True,
 [[861.0, 10, '-'],
  [123.0, 7.0, '*'],
  [133.0, 10.0, '-'],
  [140, 7.0, '-'],
  [14, 10, '*'],
  [10, 4, '+']])

In [101]:
def play_solve():
    h, v = create_game()
    status, sol = brute_speed(h.copy(), v)
    print('Hand: ' + ', '.join([str(c) for c in h]))
    print('Goal: ' + str(v))
    psol = brute_pretty_print(sol, h)
    print('Solution: ' + str(int(eval(psol))) + ' = ' + psol.replace('*', 'x'))

In [108]:
play_solve()

Hand: 10, 6, 10, 7, 8, 10, 3
Goal: 491
Solution: 491 = ((10 + 10) + (10 + 8)) x (7 + 6) - 3
